In [66]:
import geemap
import ee

In [67]:
ee.Initialize()

In [68]:
aoi = ee.FeatureCollection("projects/nb-lidar/assets/aoi_nb_south").geometry()

In [69]:
# Sentinel 2 SR

s2_sr = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2019", "2022")
    .filterBounds(aoi)
    .filter(ee.Filter.dayOfYear(91, 263))
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 1))
    .sort("CLOUDY_PIXEL_PERCENTAGE", False)
    .median()
    .clip(aoi)
)

s2_sr_vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}

In [70]:
# Senitnel 1
from src.helpers.rsd import Sentinel1Collection


s1 = (
    Sentinel1Collection()
    .filterBounds(aoi)
    .filterIWMode()
    .filterDV()
    .filterDate("2019-04-01", "2019-09-21")
    .applyEdgeMask()
    .select("V.*")
    .median()
    .clip(aoi)
)

s1_vis = {"bands": ["VV"], "min": -25, "max": 10}

# s1 processing at the collection level we will need to do data filtering, set the default date range
# then the collection need to be partitioned into 2 different datasets es and ls

In [71]:
# Alos
alos = (
    ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH")
    .filterDate("2018-01-01", "2022-01-01")
    .median()
    .select("HH")
    .clip(aoi)
)

alos_hh_vis = {"min": 0.0, "max": 10000.00}

In [72]:
Map = geemap.Map()

Map.addLayer(s1, s1_vis, "Sentinel 1")
Map.addLayer(alos, alos_hh_vis, "ALOS Palsar")
Map.addLayer(s2_sr, s2_sr_vis, "Sentinel 2 SR")
Map.centerObject(aoi, 10)
Map.addLayerControl()

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…